# Custom RAG Pipeline w/ History Implementation

This program will be adding history to our RAG pipeline, modifying history from built in RAG history retrievers to creating custom chains with the "|" operator. Creating a custom retriever allows us better control over context injection, injecting in smaller chunks with metadata chunks.

In [2]:
import os
import faiss
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

In [3]:
MODEL_NAME = "llama3.2"
llm = OllamaLLM(model= MODEL_NAME)

In [4]:
def load_docs():
    
    document_loader = []

    for root, dirs, files in os.walk("."):
        # Skip chroma_db folder
        if "faiss" in root or "git" in root:
            continue
        for file in files:
            if file.endswith(".pdf"):
                document_loader.append(file)

    return document_loader

In [5]:
document_loader = load_docs()
document_loader

['ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'Three Phase Power System Fundamentals.pdf',
 'ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf',
 'Electric Machinery Fundamentals Textbook -- Chapman.pdf',
 'ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf']

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model.save("./local_models/all-MiniLM-L6-v2")

/home/Akshay/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embedding_model ="./local_models/all-MiniLM-L6-v2" #embedding matrix model

def embed_splitting(document_loader, embedding_model):
    embeddings = HuggingFaceEmbeddings(model = embedding_model, encode_kwargs={'normalize_embeddings': True})

    doc_store = []
    for file in document_loader:
        loader = PyPDFLoader(file)
        doc = loader.load()
        doc_store += doc

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 400,
        chunk_overlap = 64
        )
    
    #Make splits
    splits = text_splitter.split_documents(doc_store)

    return embeddings, splits

In [8]:
embeddings, splits = embed_splitting(document_loader, embedding_model)

/home/Akshay/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
example_split = splits[106]
example_split

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily underst

In [10]:
print(example_split.page_content)

Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shell-type construction. 
 
 
Figure 6-6  Core-type transformer 
 
 
 
Figure 6 -7 is a schematic representation of the transformer. It will be assumed that

In [11]:
metadata = example_split.metadata
for key in metadata:
    print(f"{key}: {metadata[key]}")

producer: Microsoft® Word 2013
creator: Microsoft® Word 2013
creationdate: 2019-07-27T15:04:48+08:00
author: Ali Kharrazi
moddate: 2019-07-27T15:04:48+08:00
source: ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf
total_pages: 76
page: 51
page_label: 52


In [12]:
embeddings

HuggingFaceEmbeddings(model_name='./local_models/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
len(splits)

402

In [14]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [15]:
dim = len(embeddings.embed_query("test sentence"))
index = faiss.IndexFlatL2(dim)

if os.path.exists("faiss_index"):
    print("Loading FAISS index from disk...")
    vector_store = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
else:
    print("Building FAISS index from scratch...")
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(splits)
    vector_store.save_local("faiss_index")

Loading FAISS index from disk...


In [16]:
# create the retriever object once
semantic_retriever = vector_store.as_retriever(search_kwargs={'k': 4})

# define your function to query it
def semantic_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

# call the function with retriever and query string
results = semantic_search(semantic_retriever, "Explain transformers")

In [17]:
results[0].metadata

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2019-07-27T15:04:48+08:00',
 'author': 'Ali Kharrazi',
 'moddate': '2019-07-27T15:04:48+08:00',
 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'total_pages': 76,
 'page': 51,
 'page_label': '52'}

In [18]:
for i in range(len(results)):
    source_data = results[i].metadata["source"]
    page = results[i].metadata["page"]
    page_content = results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 51. 

 The page content is Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shel

In [19]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 4

def keyword_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

In [20]:
keyword_results = keyword_search(bm25_retriever, "Explain transformers")

In [21]:
keyword_results

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 64, 'page_label': '65'}, page_content='65 Electrical Machines and Systems                                                                                                            \n6.8 Current Transformers \nInstrument transformers are special transformers for extending the range of measur ing instruments. \nThere are two basic types: voltage transformers for measuring high voltages, and current transformers \nfor measuring high currents. Using transformers for voltage measurement is similar in principle to the \nordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current \ntransformers, on the other hand, need special consideration. These are

In [22]:
for i in range(len(keyword_results)):
    source_data = keyword_results[i].metadata["source"]
    page = keyword_results[i].metadata["page"]
    page_content = keyword_results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 64. 

 The page content is 65 Electrical Machines and Systems                                                                                                            
6.8 Current Transformers 
Instrument transformers are special transformers for extending the range of measur ing instruments. 
There are two basic types: voltage transformers for measuring high voltages, and current transformers 
for measuring high currents. Using transformers for voltage measurement is similar in principle to the 
ordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current 
transformers, on the other hand, need special consideration. These are usually toroidal transformers with 
high-quality core material. 
Figure 6-25 shows a load connected to a source. The primary of a current transformer is in series with 
the load, and the secondary 

In [23]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers= [semantic_retriever, bm25_retriever], weights = [0.6, 0.4])

In [24]:
combined_results = ensemble_retriever.invoke("Explain transformers")

In [25]:
combined_results

[Document(id='402198b9-0143-476b-beba-33c8d40d298b', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [26]:
for i in combined_results:

    print(i.metadata)

{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 50, 'page_label': '51'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 52, 'page_label': '53'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013'

In [27]:
i = 1
result_list = [combined_results[0]]
seen_pages = {combined_results[0].metadata["page_label"]}

while i < len(combined_results):
    metadata = combined_results[i].metadata
    page_label = metadata["page_label"]

    if page_label in seen_pages:
        i += 1  # You MUST increment i here
        continue

    result_list.append(combined_results[i])
    seen_pages.add(page_label)
    i += 1

In [28]:
result_list

[Document(id='402198b9-0143-476b-beba-33c8d40d298b', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [42]:
input_template = """You are an expert assistant answering based only on the provided context.

    The retrieved documents have been joined togeter separated by "Chunk_n", where n is the chunk number. Here is the context provided:

    context:
    {context}
    
    Use all relevant information above to answer the question below. If the answer isn't found in the chunks, say:
    "I cannot answer this question because the necessary information was not found in the provided documents."

    When answering, cite the **source file name** and **slide/page number** if available. Only state each **source file name** once at the end of your
    answer, along with the pages used in that file in your answer", using the metadata supplied:
    
    Metadata:
    {metadata}

    Siimilarly, the metadata is provided a dictionary separated by metadata_n, where n is the metadata corresponding to each chunk_n.
    """

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnableParallel

def content_parser(results):
    context_string = ""
    for i in range(len(results)):

        context_string += f"\nChunk_{i+1}:\n\n{results[i].page_content}\n\n\n"

    metadata_files = [[results[0].metadata["source"], results[0].metadata["page_label"]]]
    for i in range(1, len(results)):

        metadata_files.append([results[i].metadata["source"], results[i].metadata["page_label"]])

    metadata_string = ""
    for i in range(len(metadata_files)):

        metadata_string += f"metadata_{i+1}:\n\n Information source corresponding to chunk_{i+1} is {metadata_files[i][0]} page number {metadata_files[i][1]}\n\n"

    return context_string, metadata_string

chunks, metadata = content_parser(result_list)

In [31]:
print(chunks)
print(metadata)


Chunk_1:

Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shell-type construction. 
 
 
Figure 6-6  Core-type transformer 
 
 
 
Figure 6 -7 is a schematic representation of the transformer. It will be a

In [32]:
runnable_context = RunnableLambda(content_parser)
runnable_context

RunnableLambda(content_parser)

In [43]:
prompt_template = ChatPromptTemplate(
    [
        ("system", input_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [34]:
template = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI assistant. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well thanks"),
        ("human", "{user_input}")
    ]
)

In [35]:
prompt_value = template.invoke(
    {"name": "Bob",
    "user_input": "What is your name"}
)

In [44]:
chain = prompt_template | llm

In [45]:
chain.invoke(

    {"context": chunks,
    "chat_history": [],
    "metadata": metadata,
    "input": "Explain transformers"}
    
)

'Transformers are electrical devices that transfer electrical energy from one circuit or system to another through electromagnetic induction. They consist of two coils of wire, known as the primary and secondary coils, which are wrapped around a common core.\n\nThe primary coil receives an alternating current (AC) input, which induces a magnetic field in the core. This magnetic field then induces a voltage in the secondary coil, causing electrical energy to be transferred from the primary circuit to the secondary circuit.\n\nTransformers can operate as either step-up or step-down devices, depending on the ratio of turns between the primary and secondary coils. A higher ratio of turns results in a step-down transformation, while a lower ratio results in a step-up transformation.\n\nThe core material used in transformers can affect their performance. High-quality core materials are often used to minimize energy losses and improve efficiency.\n\nIn instrument transformers, such as current

In [39]:
chain

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input', 'metadata'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk

In [ ]:
prompt_template

In [ ]:
chain = (
    context_parser(input) | runnable_context
    metadata_parser(input) | runnable_metadata,
    prompt_template |
    llm
)

In [ ]:
answer1 = chain.invoke("Explain transformers")